# 2 start pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

## login AKS

In [ ]:
az login --tenant ${TENANT}

## create wildcard TLS cert files using Let's Encrypt

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "docker run -it -v ${PJ_ROOT}/secrets:/etc/letsencrypt certbot/certbot certonly --manual --domain *.${DOMAIN} --email ${EMAIL} --no-eff-email --agree-tos --manual-public-ip-logging-ok --preferred-challenges dns-01 --server https://acme-v02.api.letsencrypt.org/directory"

_Outside of this notebook_
1. wait until to add a DNS TXT Record like below.

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

_Outside of this notebook_
1. in order to continue creating cert file process, press enter key in the opened terminal.
1. confirm to be displayed the below message at the opened terminal.
```
Congratulations! Your certificate and chain have been saved at:
```

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt remove-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

## start RabbitMQ cluster on AKS
[rabbitmq](https://www.rabbitmq.com/)

### register cert files to kubernetes secrets

In [ ]:
kubectl create secret generic rabbitmq-certifications --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/cert.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start RabbitMQ

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-rbac.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-services.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-statefulset.yaml

In [ ]:
kubectl get pods -l app=rabbitmq

example)

```
NAME         READY     STATUS    RESTARTS   AGE
rabbitmq-0   1/1       Running   0          6m
rabbitmq-1   1/1       Running   0          5m
rabbitmq-2   1/1       Running   0          4m
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl cluster_status

example)
```
Cluster status of node rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local ...
[{nodes,[{disc,['rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local']}]},
 {running_nodes,['rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local']},
 {cluster_name,<<"rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local">>},
 {partitions,[]},
 {alarms,[{'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',[]}]}]
```

### change password of `guest`

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
ros	[]
raspberrypi	[]
guest	[administrator]
iotagent	[]
```

### register DNS A Record for RabbitMQ

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=rabbitmq -l service=mqtts

expected)
```
NAME             TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)          AGE
rabbitmq-mqtts   LoadBalancer   10.0.96.7    XX.XX.XX.XX   8883:30299/TCP   21m
```

In [ ]:
MQTTS_IPADDR=$(kubectl get services -l app=rabbitmq -l service=mqtts -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}')
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "mqtt" --ipv4-address "${MQTTS_IPADDR}"

In [ ]:
nslookup mqtt.${DOMAIN}

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on AKS
[mongodb](https://www.mongodb.com/)

create 'mongodb' takes a few minutes.

In [ ]:
kubectl apply -f mongodb/mongodb-cluster-azure.yaml

In [ ]:
kubectl get PersistentVolumeClaims -l app=mongodb

example)
```
NAME                              STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS      AGE
mongodb-storage-claim-mongodb-0   Bound     pvc-386ac3ba-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   5m
mongodb-storage-claim-mongodb-1   Bound     pvc-88455199-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   3m
mongodb-storage-claim-mongodb-2   Bound     pvc-b74ec32c-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   2m
```

In [ ]:
kubectl get pods -l app=mongodb

example)
```
NAME        READY     STATUS    RESTARTS   AGE
mongodb-0   2/2       Running   0          6m
mongodb-1   2/2       Running   0          4m
mongodb-2   2/2       Running   0          2m
```

In [ ]:
kubectl get services -l app=mongodb

example)
```
NAME      TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb   ClusterIP   None         <none>        27017/TCP   6m
```

In [ ]:
kubectl exec mongodb-0 -c mongodb -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.6
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on AKS

[ambassador](https://www.getambassador.io/)

### register cert files of ambassador to kubernetes secrets

In [ ]:
kubectl create secret tls ambassador-certs --cert=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --key=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-azure-services.yaml

In [ ]:
kubectl apply -f ambassador/ambassador-deployment.yaml

In [ ]:
kubectl get pods -l service=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   1/1       Running   0          39s
ambassador-cf99ff6bb-8kht7   1/1       Running   0          39s
ambassador-cf99ff6bb-lv2tk   1/1       Running   0          39s
```

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l service=ambassador

example)
```
NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)                      AGE
ambassador   LoadBalancer   10.0.191.59   YY.YY.YY.YY   443:30357/TCP,80:32755/TCP   4m
```

### register DNS A Record for ambassador

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l service=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "api" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup api.${DOMAIN}

In [ ]:
curl -i https://api.${DOMAIN}

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```

## start authorization & authentication service on AKS

### create `secrets/auth-tokens.json`
example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "chsNJIuPM9JJamcfEPHwvYVaPWxVBuHB",
          "allowed_paths": [
            "^/orion/.*$",
            "^/idas/.*$",
            "^/sth/.*$"
          ]
        },
        {
          "token": "IfFT40fIGviZxZQmGwA5JxGwRHAIqsv7",
          "allowed_paths": [
            "^/visualizer/positions/$"
          ]
        }
      ],
      "basic_auths": [
        {
          "username": "user1",
          "password": "q71UiXsmtWNtciPg",
          "allowed_paths": [
            "/controller/web/"
          ]
        },
        {
          "username": "visualizer",
          "password": "1Fz9WzheDYf7fpxh",
          "allowed_paths": [
            "/visualizer/locus/"
          ]
        }
      ],
      "no_auths": {
        "allowed_paths": [
          "^.*/static/.*$"
        ]
      }
    }
  },
  {
    "host": "wirecloud\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": [
          "^/.*$"
        ]
      }
    }
  },
  {
    "host": "proxy\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "8fG2ZJGArqFXKnZm1Cv6NjyJp5uvGRA2",
          "allowed_paths": [
            "^/events/$"
          ]
        }
      ],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": []
      }
    }
  }
]
```

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": "api\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/sth/.*$"]
                }, {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/visualizer/positions/$"]
                }
            ],
            "basic_auths": [
                {
                    "username": "user1",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/controller/web/"]
                }, {
                    "username": "visualizer",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/visualizer/locus/"]
                }
            ],
            "no_auths": {
                "allowed_paths": ["^.*/static/.*$"]
            }
        }
    },
    {
        "host": "wirecloud\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": ["^/.*$"]
            }
        }
    },
    {
        "host": "proxy\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/events/$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }
]
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start fiware-ambassador-auth

In [ ]:
kubectl apply -f ambassador/fiware-ambassador-auth.yaml

In [ ]:
kubectl get pods -l pod=ambassador-auth

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
ambassador-auth-6fffdbd9c9-7kkpr   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-qxw6m   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-sdn5b   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l service=ambassador-auth

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
ambassador-auth   ClusterIP   10.0.129.102   <none>        3000/TCP   2m
```

In [ ]:
curl -i https://api.${DOMAIN}/

example)
```
HTTP/1.1 401 Unauthorized
content-type: application/json; charset=utf-8
www-authenticate: Bearer realm="token_required"
date: Thu, 12 Jul 2018 02:21:24 GMT
content-length: 60
x-envoy-upstream-service-time: 2
server: envoy

{"authorized":false,"error":"missing Header: authorization"}
```

## start fiware orion on AKS
[fiware orion](https://catalogue-server.fiware.org/enablers/publishsubscribe-context-broker-orion-context-broker)

In [ ]:
kubectl apply -f orion/orion.yaml

In [ ]:
kubectl get pods -l app=orion

example)
```
NAME                     READY     STATUS    RESTARTS   AGE
orion-54f5cdcb5d-d2pt5   1/1       Running   0          56s
orion-54f5cdcb5d-hv274   1/1       Running   0          56s
orion-54f5cdcb5d-xbnx2   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l app=orion

example)
```
NAME      TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
orion     ClusterIP   10.0.44.126   <none>        1026/TCP   1m
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 200 OK
content-length: 2
content-type: application/json
fiware-correlator: 4731eb48-4dc1-11e8-b1a2-0a580af4010a
date: Wed, 02 May 2018 04:28:35 GMT
x-envoy-upstream-service-time: 5
server: envoy

[]
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware idas(iotagent-ul) on AKS
[fiware IDAS(iotagent-ul)](https://catalogue-server.fiware.org/enablers/backend-device-management-idas)

### build and push the idas conatiner image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/iotagent-ul:290a1fa idas/iotagent-ul/

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/iotagent-ul:290a1fa

expected)
```
The push refers to repository [ccfdacr.azurecr.io/tech-sketch/iotagent-ul]

1f141107: Preparing 
e3e1d3a9: Preparing 
a102cc1f: Preparing 
290a1fa: digest: sha256:468a6c2e9aaf6d47a4e21d95bbdb9f8c2df088e876e0846ceaba09cca5fc2d13 size: 1163
```

In [ ]:
az acr repository list --name ${ACR_NAME} --output table

example)
```
Result
---------------------------------
tech-sketch/iotagent-ul
```

### start idas

In [ ]:
env IOTA_PASSWORD=${MQTT__iotagent} envsubst < idas/config.js > /tmp/config.js
kubectl create secret generic iotagent-config --from-file /tmp/config.js
rm /tmp/config.js

In [ ]:
envsubst < idas/iotagent-ul.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=iotagent-ul

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
iotagent-ul-79685b64bf-8krps   1/1       Running   0          3m
iotagent-ul-79685b64bf-m6nlg   1/1       Running   0          3m
iotagent-ul-79685b64bf-mjpbl   1/1       Running   0          3m
```

In [ ]:
kubectl get services -l app=iotagent-ul

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
iotagent-ul   ClusterIP   10.0.180.155   <none>        4041/TCP,7896/TCP   43s
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: /*" https://api.${DOMAIN}/idas/ul20/manage/iot/services/

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
fiware-correlator: c114fc5e-b4a2-40f6-b7fe-1d68369784e5
content-type: application/json; charset=utf-8
content-length: 25
etag: W/"19-WMYe0U6ocKhQjp+oaVnMHLdbylc"
date: Wed, 02 May 2018 06:16:18 GMT
x-envoy-upstream-service-time: 9
server: envoy

{"count":0,"services":[]}
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware cygnus (mongodb sink & sth sink) on AKS
[fiware cygnus](https://catalogue-server.fiware.org/enablers/cygnus)

* build fiware-cygnus for RoboticBase

```bash
fiware-cygnus/docker/cygnus-ngsi$ docker build --build-arg GIT_URL_CYGNUS=https://github.com/tech-sketch/fiware-cygnus.git --build-arg GIT_REV_CYGNUS=feature/1516_create_elasticsearchsink -t techsketch/fiware-cygnus:1.9.0.elasticsearch .
fiware-cygnus/docker/cygnus-ngsi$ docker push techsketch/fiware-cygnus:1.9.0.elasticsearch
```

### start cygnus-mongo

In [ ]:
kubectl apply -f cygnus/cygnus-mongo.yaml

In [ ]:
kubectl get pods -l app=cygnus-mongo

example)
```
NAME                            READY     STATUS    RESTARTS   AGE
cygnus-mongo-588cb8cb46-42bcn   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-k9cr2   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-phvst   1/1       Running   0          1m
```

In [ ]:
kubectl get services -l app=cygnus-mongo

example)
```
NAME           TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)             AGE
cygnus-mongo   ClusterIP   10.0.10.63   <none>        5050/TCP,8081/TCP   1m
```

## start fiware STH-Comet on AKS

In [ ]:
kubectl apply -f sth-comet/sth-service.yaml

In [ ]:
kubectl apply -f sth-comet/sth-deployment.yaml

In [ ]:
kubectl get services -l app=sth

example)
```
NAME   TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)    AGE
sth    ClusterIP   10.0.49.77   <none>        8666/TCP   30s
```

In [ ]:
kubectl get pods -l app=sth

example)
```
NAME                   READY   STATUS    RESTARTS   AGE
sth-859bd88557-6rxdq   1/1     Running   0          46s
sth-859bd88557-jpsds   1/1     Running   0          46s
sth-859bd88557-pk7tm   1/1     Running   0          46s
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: test" -H "Fiware-ServicePath: test" https://api.${DOMAIN}/sth/STH/v1/contextEntities/type/t/id/i/attributes/a?lastN=1

example)
```
HTTP/1.1 200 OK
fiware-correlator: b5f27bd3-293f-449d-9467-f5eb23dd1fca
content-type: application/json; charset=utf-8
cache-control: no-cache
content-length: 169
vary: accept-encoding
accept-ranges: bytes
date: Thu, 24 Jan 2019 07:05:19 GMT
x-envoy-upstream-service-time: 4
server: envoy

{"contextResponses":[{"contextElement":{"attributes":[{"name":"a","values":[]}],"id":"i","isPattern":false,"type":"t"},"statusCode":{"code":"200","reasonPhrase":"OK"}}]}
```

## start mcrouter (memcached cluster + mcrouter)

In [ ]:
helm install stable/mcrouter --name=mcrouter -f mcrouter/values.yaml

In [ ]:
kubectl expose service mcrouter-mcrouter --port 5000 --target-port 5000 --name mcrouter --type ClusterIP

In [ ]:
kubectl get configmap -l release=mcrouter

example)
```
NAME                DATA   AGE
mcrouter-mcrouter   1      12m
```

In [ ]:
kubectl get statefulsets -l release=mcrouter

example)
```
NAME                 DESIRED   CURRENT   AGE
mcrouter-mcrouter    3         3         12m
mcrouter-memcached   3         3         12m
```

In [ ]:
kubectl get pods -l release=mcrouter

example)
```
NAME                   READY   STATUS    RESTARTS   AGE
mcrouter-mcrouter-0    1/1     Running   0          13m
mcrouter-mcrouter-1    1/1     Running   0          13m
mcrouter-mcrouter-2    1/1     Running   0          13m
mcrouter-memcached-0   1/1     Running   0          13m
mcrouter-memcached-1   1/1     Running   0          13m
mcrouter-memcached-2   1/1     Running   0          13m
```

In [ ]:
kubectl get services -l release=mcrouter

example)
```
NAME                 TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)     AGE
mcrouter             ClusterIP   10.0.95.191   <none>        5000/TCP    8m
mcrouter-mcrouter    ClusterIP   None          <none>        5000/TCP    14m
mcrouter-memcached   ClusterIP   None          <none>        11211/TCP   14m
```

## start stolon (postgreSQL cluster) on AKS

1. edit superuser password & replication password of `stolon/values.yaml`

### start stolon (postgreSQL cluster) using Helm

In [ ]:
helm install --name stolon -f stolon/values.yaml stable/stolon

In [ ]:
kubectl get services -l app=stolon

example)
```
NAME                     TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
stolon-keeper-headless   ClusterIP   None           <none>        5432/TCP   51m
stolon-proxy             ClusterIP   10.0.172.206   <none>        5432/TCP   51m
```

In [ ]:
kubectl get deployments -l app=stolon

example)
```
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
stolon-proxy      2         2         2            2           52m
stolon-sentinel   2         2         2            2           52m
```

In [ ]:
kubectl get statefulsets -l app=stolon

example)
```
NAME            DESIRED   CURRENT   AGE
stolon-keeper   2         2         53m
```

In [ ]:
kubectl get jobs -l app=stolon

example)
```
NAME                    DESIRED   SUCCESSFUL   AGE
stolon-create-cluster   1         1            54m
```

In [ ]:
kubectl get pvc -l app=stolon

example)
```
NAME                   STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS      AGE
data-stolon-keeper-0   Bound    pvc-78e58e79-1f76-11e9-a070-32bf9bf468a2   10Gi       RWO            managed-premium   54m
data-stolon-keeper-1   Bound    pvc-8e02ca72-1f76-11e9-a070-32bf9bf468a2   10Gi       RWO            managed-premium   54m
```

In [ ]:
kubectl get pods -l app=stolon

example)
```
NAME                               READY   STATUS      RESTARTS   AGE
stolon-create-cluster-tw7jl        0/1     Completed   0          55m
stolon-keeper-0                    1/1     Running     0          55m
stolon-keeper-1                    1/1     Running     0          54m
stolon-proxy-8645999c59-bqkjr      1/1     Running     0          55m
stolon-proxy-8645999c59-fb9mw      1/1     Running     0          55m
stolon-sentinel-58dfdf649c-565gd   1/1     Running     0          55m
stolon-sentinel-58dfdf649c-msg9r   1/1     Running     0          55m
```

### create user and database on PostgreSQL for WireCloud

In [ ]:
PGPASSWORD=$(grep 'superuserPassword:' ${PJ_ROOT}/stolon/values.yaml | awk '{print $2}')
kubectl exec -it stolon-keeper-0 -- /bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 postgres -U stolon -c \"DROP DATABASE IF EXISTS wirecloud;\""
kubectl exec -it stolon-keeper-0 -- /bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 postgres -U stolon -c \"DROP USER IF EXISTS wirecloud;\""
kubectl exec -it stolon-keeper-0 -- /bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 postgres -U stolon -c \"CREATE DATABASE wirecloud;\""
kubectl exec -it stolon-keeper-0 -- /bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 postgres -U stolon -c \"CREATE USER wirecloud WITH PASSWORD '${PGPASSWORD}'\""
kubectl exec -it stolon-keeper-0 -- /bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 postgres -U stolon -c \"GRANT ALL PRIVILEGES ON DATABASE wirecloud TO wirecloud;\""

### confirm wirecloud database

In [ ]:
PGPASSWORD=$(grep 'superuserPassword:' ${PJ_ROOT}/stolon/values.yaml | awk '{print $2}')
kubectl exec -it stolon-keeper-0 -- bin/bash -c "PGPASSWORD=${PGPASSWORD} psql --host stolon-proxy --port 5432 wirecloud -U wirecloud -c 'select now()'"

example)
```
              now              
-------------------------------
 2019-01-24 03:15:39.513721+00
(1 row)
```

## start fiware wirecloud

### create shared persistent volume

In [ ]:
NODERG=$(az aks show --resource-group ${AKS_RG} --name ${AKS_NAME} --query nodeResourceGroup -o tsv)
az storage account create --resource-group ${NODERG} --name ${AKS_NAME}storageaccount --sku Standard_LRS

In [ ]:
envsubst < ${PJ_ROOT}/wirecloud/shared-volume-azure.yaml | kubectl apply -f -

In [ ]:
kubectl get pvc -l app=wirecloud

example)
```
NAME                      STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
wirecloud-shared-data     Bound    pvc-a24a26b9-1fb6-11e9-a070-32bf9bf468a2   10Gi       RWX            azurefile      13s
wirecloud-shared-static   Bound    pvc-a252c2fe-1fb6-11e9-a070-32bf9bf468a2   10Gi       RWX            azurefile      13s
```

### start wirecloud

In [ ]:
kubectl apply -f wirecloud/wirecloud-service.yaml

In [ ]:
kubectl apply -f wirecloud/wirecloud-nginx-conf.yaml

In [ ]:
export PGPASSWORD=$(grep 'superuserPassword:' ${PJ_ROOT}/stolon/values.yaml | awk '{print $2}')
envsubst < wirecloud/wirecloud-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get services -l app=wirecloud

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
wirecloud   ClusterIP   10.0.130.72   <none>        8888/TCP   25m
```

In [ ]:
kubectl get pods -l app=wirecloud

example)
```
NAME                         READY   STATUS    RESTARTS   AGE
wirecloud-679cb9dffb-6q5bt   2/2     Running   0          7m
wirecloud-679cb9dffb-8ncqk   2/2     Running   0          7m
wirecloud-679cb9dffb-cqmqh   2/2     Running   0          7m
```

### create user of WireCloud

In [ ]:
WIRECLOUD_USER=wirecloud-user1
WIRECLOUD_PASSWORD=$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)
echo $WIRECLOUD_USER:$WIRECLOUD_PASSWORD
kubectl exec -it $(kubectl get pods -l app=wirecloud -o name | awk -F '/' '{print $2}' | awk 'NR==1') -c wirecloud-django -- python manage.py shell -c "from django.contrib.auth import get_user_model; User = get_user_model(); User.objects.create_user('${WIRECLOUD_USER}', 'wirecloud.admin@${DOMAIN}', '${WIRECLOUD_PASSWORD}')"

## start WireCloud-Proxy

In [ ]:
kubectl apply -f proxy/proxy-service.yaml

In [ ]:
kubectl apply -f proxy/proxy-deployment.yaml

In [ ]:
kubectl get services -l app=proxy

example)
```
NAME    TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
proxy   ClusterIP   10.0.148.124   <none>        3000/TCP   33s
```

In [ ]:
kubectl get pods -l app=proxy

example)
```
NAME                    READY   STATUS    RESTARTS   AGE
proxy-cb789bcf8-9dkbd   1/1     Running   0          41s
proxy-cb789bcf8-qwvwd   1/1     Running   0          41s
proxy-cb789bcf8-xsrpw   1/1     Running   0          41s
```

In [ ]:
TOKEN=$(cat secrets/auth-tokens.json | jq '.[2].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" https://proxy.cloudconductor.jp/events/ -G --data-urlencode fiwareService=test --data-urlencode fiwareServicePath=/test --data-urlencode typePattern=t --data-urlencode idPattern=i --data-urlencode attrs=a

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
content-type: text/event-stream
cache-control: no-cache
date: Thu, 24 Jan 2019 07:41:37 GMT
x-envoy-upstream-service-time: 22
server: envoy
transfer-encoding: chunked


:
```

## start command proxy service on AKS

In [ ]:
envsubst < controller/fiware-cmd-proxy.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l pod=cmd-proxy

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
cmd-proxy-d7cbc6dfb-6hnsp   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-v6rrx   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-zf2fd   1/1       Running   0          46s
```

In [ ]:
kubectl get services -l service=cmd-proxy

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
cmd-proxy   ClusterIP   10.0.172.56   <none>        8888/TCP   1m
```

## start robot visualization service on AKS

In [ ]:
export MONGODB_DATABASE="sth_${FIWARE_SERVICE}"
export MONGODB_COLLECTION="sth_${ROBOT_SERVICEPATH}_${ROBOT_ID}_${ROBOT_TYPE}"
env BEARER_AUTH=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens | map(select(.allowed_paths[] | contains ("^/visualizer/positions/$"))) | .[0].token' -r) envsubst < controller/fiware-robot-visualization.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l pod=robot-visualization

example)
```
NAME                                   READY     STATUS    RESTARTS   AGE
robot-visualization-7d69bcdff7-g7p98   1/1       Running   0          13m
robot-visualization-7d69bcdff7-k6mb9   1/1       Running   0          13m
robot-visualization-7d69bcdff7-txsqn   1/1       Running   0          13m
```

In [ ]:
kubectl get services -l service=robot-visualization

example)
```
NAME                  TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
robot-visualization   ClusterIP   10.0.112.72   <none>        8888/TCP   13m
```